# CS 109A/STAT 121A/AC 209A/CSCI E-109A

## Standard Section 5: Partial Components Analysis (PCA) Fitting and Model Selection

**Harvard University**<br/>
**Fall 2017**<br/>
**Section Leaders: Albert Wu, Nathaniel Burbank<br/>**
**Instructors: Pavlos Protopapas, Kevin Rader, Rahul Dave, Margo Levine** 

<center>**Download this notebook from the CS109 repo or here:**</center>
<center>**http://bit.ly/109_S5sol**</center>

For this section, our goal is to review and further our understanding of the Partial Components Analysis (PCA) model. PCA is highly effective in applications to high dimensionional datasets, which we will use here. Specifically, this section is designed to help us answer Homework 4, part (h). 

Specifically, we will: 
    
    1. Review the basics of Partial Components Analysis and hone our intution
    2. Discuss implementation of PCA within Python and coding issues to keep in mind
    3. Use the principles of model selection we have learned in lecture to find a "best" PCA feature set.
    4. Compare our PCA model with other models we have fit in labs and lecture and discuss coefficient meanings.


For this section we will be using the following packages:

In [1]:
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)
pd.set_option('display.width', 1000)
pd.set_option('display.notebook_repr_html', True)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.api import OLS
from statsmodels.api import add_constant
from statsmodels.regression.linear_model import RegressionResults
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
# Note --  Requires sklearn version .18 or higher  

from sklearn.metrics import r2_score
from collections import Counter
sns.set(style="ticks")
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

matplotlib.rcParams['figure.figsize'] = (13.0, 6.0)

assert(sys.version_info.major==3),print(sys.version)
# Python 3 or higher is required

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Part (1): Load in our data and conduct basic EDA


We will first load in our dataset below and look at the first few rows. Then, we use the describe function to get a sense of the data.

In [2]:
crime_df = pd.read_csv('https://raw.githubusercontent.com/albertw1/data/master/Crime.csv').drop(['Date', 'Year'], axis=1)
crime_df.head()

,Incidence,Temp,Dewpoint,Windspeed,Pressure,Precipitation,TMAX_C,TMIN_C,Month,DOW,Weekend,Season
0,182,26.444444,15.944444,8.5,1008.5,0.0,31.722222,21.722222,7,Sunday,1,Summer
1,295,24.333333,9.666667,7.9,1012.2,0.0,31.722222,20.000000,7,Monday,0,Summer
2,267,22.722222,9.666667,5.5,1016.0,0.0,28.888889,18.277778,7,Tuesday,0,Summer
3,250,23.166667,14.500000,6.7,1020.1,0.0,28.277778,18.277778,7,Wednesday,0,Summer
4,259,24.500000,15.611111,8.6,1021.3,0.0,30.000000,18.888889,7,Thursday,0,Summer


In [454]:
crime_df.describe()

,Incidence,Temp,Dewpoint,Windspeed,Pressure,Precipitation,TMAX_C,TMIN_C,Month,Weekend
count,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000
mean,238.107763,10.583815,3.713090,9.168767,1016.463562,0.114913,16.309082,5.841755,6.526027,0.285845
std,31.539843,9.913817,10.848571,3.269969,7.632274,0.303409,10.445645,9.806276,3.449427,0.452022
min,115.000000,-14.222222,-24.277778,2.200000,987.400000,0.000000,-10.000000,-19.388889,1.000000,0.000000
25%,219.000000,2.500000,-4.055556,6.900000,1011.300000,0.000000,7.222222,-1.111111,4.000000,0.000000
50%,240.000000,11.333333,4.722222,8.700000,1016.400000,0.000000,17.222222,6.722222,7.000000,0.000000
75%,259.000000,19.111111,12.888889,11.000000,1021.400000,0.055000,25.611111,15.000000,10.000000,1.000000
max,349.000000,30.944444,22.444444,25.800000,1040.400000,3.540000,37.222222,26.111111,12.000000,1.000000


Convert the columns that are categorical variables into dummy variables by one-hot encoding.

In [455]:
categorical_columns = ['Month', 'Weekend', 'Season', 'DOW']
numerical_columns = ['Temp', 'Dewpoint', 'Windspeed', 'Pressure', 'Precipitation', 'TMAX_C', 'TMIN_C']
crime_df = pd.get_dummies(crime_df, columns=categorical_columns, drop_first=True)
crime_df.head()

,Incidence,Temp,Dewpoint,Windspeed,Pressure,Precipitation,TMAX_C,TMIN_C,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Weekend_1,Season_Spring,Season_Summer,Season_Winter,DOW_Monday,DOW_Saturday,DOW_Sunday,DOW_Thursday,DOW_Tuesday,DOW_Wednesday
0,182,26.444444,15.944444,8.5,1008.5,0.0,31.722222,21.722222,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0
1,295,24.333333,9.666667,7.9,1012.2,0.0,31.722222,20.000000,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,267,22.722222,9.666667,5.5,1016.0,0.0,28.888889,18.277778,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
3,250,23.166667,14.500000,6.7,1020.1,0.0,28.277778,18.277778,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,259,24.500000,15.611111,8.6,1021.3,0.0,30.000000,18.888889,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


Now, let's split this dataset up into a testing and training set.

In [496]:
train, test =  train_test_split(crime_df, test_size=.2, random_state=123)
train.shape,test.shape

((876, 12), (219, 12))

Now let us standarize the numerical variables only.

In [497]:
mean = train[numerical_columns].mean()
std = train[numerical_columns].std()

train[numerical_columns] = (train[numerical_columns] - mean)/std
test[numerical_columns] = (test[numerical_columns] - mean)/std

Now to let us subset and create convenient formats for parts in train and test

In [457]:
all_predictors = ['Temp','Dewpoint','Windspeed','Pressure','Precipitation','TMAX_C','TMIN_C','Month_2','Month_3','Month_4','Month_5','Month_6','Month_7','Month_8','Month_9','Month_10','Month_11','Month_12','Weekend_1','Season_Spring','Season_Summer','Season_Winter','DOW_Monday','DOW_Saturday','DOW_Sunday','DOW_Thursday','DOW_Tuesday','DOW_Wednesday']

X_train_df = train[all_predictors]
X_test_df  = test[all_predictors]
X_train_np = train[all_predictors].values
X_test_np  = test[all_predictors].values
y_train = train['Incidence'].values
y_test = test['Incidence'].values
y_test.shape

(219,)

## Part (2): Use subset selection to fit a linear regression model


Let's use the forward/backward subset selection method from HW 3 to fit a linear regression model on the data.

In [498]:
def step_forwards_backwards(df, y_val, direction='forward'):
    
    assert direction in ['forward', 'backward']
    
    y = y_val.reshape(-1,1)
    
    predictors = set(df.columns)
    selected_predictors = set() if direction=='forward' else set(predictors)
    
    n = df.shape[0]
    best_bic = np.inf
    
    best_bics = []
    best_models = []
    
    if direction == 'forward':
        X = np.ones(n).reshape(-1,1)
        X = np.concatenate([X, df[list(selected_predictors)].values], axis=1)
        while (True):
            
            possible_bic_scores = []
            possible_predictors = list(selected_predictors ^ predictors)
            
            if len(possible_predictors) == 0:
                break
                
            for predictor in possible_predictors:
                
                x_temp = np.concatenate([X, df[predictor].values.reshape(-1,1)], axis=1)
                model = OLS(endog=y, exog=x_temp).fit()
                bic = model.bic
                possible_bic_scores.append(bic)
                
            best_predictor_ix = np.argmin(possible_bic_scores)
            best_predictor = possible_predictors[best_predictor_ix]
            
            best_bic = np.min(possible_bic_scores)
            best_bics.append(best_bic)
            
            selected_predictors.add(best_predictor)            
            X = np.concatenate([X, df[best_predictor].values.reshape(-1,1)], axis=1)
            best_models.append(list(selected_predictors))

    else:

        while (True):
            possible_bic_scores = []
            possible_predictors = list(selected_predictors)

            if len(possible_predictors) == 0:
                break

            for predictor in possible_predictors:
                X = np.concatenate([np.ones(n).reshape(-1,1), df[list(selected_predictors - set([predictor]))].values], axis=1)
                model = OLS(endog=y, exog=X).fit()
                bic = model.bic
                possible_bic_scores.append(bic)

            best_predictor_ix = np.argmin(possible_bic_scores)
            best_predictor = possible_predictors[best_predictor_ix] 

            best_bic = possible_bic_scores[best_predictor_ix]
            selected_predictors.discard(best_predictor)
            
            best_bics.append(best_bic)
            best_models.append(list(selected_predictors))
            
    index_of_best_bic = np.argmin(best_bics)

    return best_models[index_of_best_bic]

Let's run the subset selection function and see which variables were included in the best model:

In [499]:
predictors_forward = step_forwards_backwards(X_train_df, y_train, direction='forward')
predictors_forward

['DOW_Sunday',
 'DOW_Monday',
 'DOW_Saturday',
 'Windspeed',
 'Temp',
 'Month_12',
 'DOW_Wednesday',
 'DOW_Thursday',
 'Month_9',
 'Precipitation',
 'DOW_Tuesday']

In [500]:
predictors_backward = step_forwards_backwards(X_train_df, y_train, direction='backward')
predictors_backward

['DOW_Wednesday',
 'DOW_Monday',
 'DOW_Saturday',
 'Windspeed',
 'Month_9',
 'DOW_Thursday',
 'DOW_Tuesday',
 'Weekend_1',
 'Dewpoint',
 'DOW_Sunday',
 'Month_5',
 'Temp']

Based on these variables, we can see what the R-squared values are for our training and testing sets.

In [501]:
X = sm.add_constant(X_train_df[predictors_backward])
X_test = sm.add_constant(X_test_df[predictors_backward])
y = train['Incidence'].values.reshape(-1,1)

model = OLS(endog=y, exog=X)
result = model.fit()

y_hat_train = result.predict()
y_hat_test = result.predict(exog=X_test)

print('Backward Selection Training R2 = ', r2_score(y_train, y_hat_train))
print('Backward Selection Testing R2 = ', r2_score(y_test, y_hat_test))

Backward Selection Training R2 =  0.418781376528
Backward Selection Testing R2 =  0.356416351464


In [502]:
X = sm.add_constant(X_train_df[predictors_forward])
X_test = sm.add_constant(X_test_df[predictors_forward])
y = train['Incidence'].values.reshape(-1,1)

model = OLS(endog=y, exog=X)
result = model.fit()

y_hat_train = result.predict()
y_hat_test = result.predict(exog=X_test)

print('Forward Selection Training R2 = ', r2_score(y_train, y_hat_train))
print('Forward Selection Testing R2 = ', r2_score(y_test, y_hat_test))

Forward Selection Training R2 =  0.419038207984
Forward Selection Testing R2 =  0.357321097746


## Part (3): Create a data frame with continuous predictors taken to polynomial power

Now, we will work with an example where we will manually take numeric predictors and take them to a polynomial power. The next step is a simple example to help you see how we can do this.

In [405]:
np.hstack((np.array([[1, 2,3], [4, 5,6],[7,8,9],[10,11,12]])**(i+1) for i in range(3)))

array([[   1,    2,    3,    1,    4,    9,    1,    8,   27],
       [   4,    5,    6,   16,   25,   36,   64,  125,  216],
       [   7,    8,    9,   49,   64,   81,  343,  512,  729],
       [  10,   11,   12,  100,  121,  144, 1000, 1331, 1728]])

We want to create a data frame with the continuous predictors taken up to a power 3, while keeping the rest of the categorical predictors the same. 

In [503]:
X_train_numerical_powers = np.hstack((X_train_df[numerical_columns]**(i+1) for i in range(3)))

print('Number of Total Predictors with Continuous Polynomial Terms Added is', X_train_num_powers.shape[1])

Number of Total Predictors with Continuous Polynomial Terms Added is 21


In [507]:
X_train_np_powers = np.concatenate((X_train_numerical_powers,X_train_df.drop(numerical_columns, axis=1)),axis=1)
X_train_df_powers = pd.DataFrame(X_train_np_powers)
newcolname = ['Temp', 'Dewpoint', 'Windspeed', 'Pressure', 'Precipitation', 'TMAX_C', 'TMIN_C', 'Temp^2', 'Dewpoint^2', 'Windspeed^2', 'Pressure^2', 'Precipitation^2', 'TMAX_C^2', 'TMIN_^2','Temp^3', 'Dewpoint^3', 'Windspeed^3', 'Pressure^3', 'Precipitation^3', 'TMAX_C^3', 'TMIN_C^3'] + list(X_train_df.drop(numerical_columns, axis=1))
X_train_df_powers.columns = newcolname
X_train_df_powers                                          
                             

,Temp,Dewpoint,Windspeed,Pressure,Precipitation,TMAX_C,TMIN_C,Temp^2,Dewpoint^2,Windspeed^2,Pressure^2,Precipitation^2,TMAX_C^2,TMIN_^2,Temp^3,Dewpoint^3,Windspeed^3,Pressure^3,Precipitation^3,TMAX_C^3,TMIN_C^3,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Weekend_1,Season_Spring,Season_Summer,Season_Winter,DOW_Monday,DOW_Saturday,DOW_Sunday,DOW_Thursday,DOW_Tuesday,DOW_Wednesday
0,24.888889,21.166667,7.2,1014.0,0.54,29.388889,19.388889,619.456790,448.027778,51.84,1028196.00,0.2916,863.706790,375.929012,15417.591223,9483.254634,373.248,1.042591e+09,0.157464,25383.382890,7288.845852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-4.333333,-15.833333,12.8,1016.8,0.01,3.888889,-8.888889,18.777778,250.694444,163.84,1033882.24,0.0001,15.123457,79.012346,-81.370370,-3969.328701,2097.152,1.051251e+09,0.000001,58.813443,-702.331962,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6.055556,3.888889,5.6,1015.9,0.00,10.000000,3.888889,36.669753,15.123457,31.36,1032052.81,0.0000,100.000000,15.123457,222.055727,58.813443,175.616,1.048462e+09,0.000000,1000.000000,58.813443,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-6.722222,-11.666667,14.3,1014.0,0.25,-0.611111,-12.222222,45.188272,136.111111,204.49,1028196.00,0.0625,0.373457,149.382716,-303.765604,-1587.962964,2924.207,1.042591e+09,0.015625,-0.228224,-1825.788751,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,9.666667,2.444444,8.8,1027.6,0.00,18.888889,7.222222,93.444444,5.975309,77.44,1055961.76,0.0000,356.790123,52.160494,903.296296,14.606310,681.472,1.085106e+09,0.000000,6739.369000,376.714678,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-0.277778,-12.722222,15.8,1005.4,0.08,3.277778,-2.222222,0.077160,161.854938,249.64,1010829.16,0.0064,10.743827,4.938272,-0.021433,-2059.154491,3944.312,1.016288e+09,0.000512,35.215878,-10.973937,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4.611111,-3.666667,11.9,1023.4,0.09,10.611111,1.111111,21.262346,13.444444,141.61,1047347.56,0.0081,112.595679,1.234568,98.043038,-49.296296,1685.159,1.071855e+09,0.000729,1194.765260,1.371742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
7,2.611111,-8.222222,12.1,1013.6,0.00,8.888889,-4.388889,6.817901,67.604938,146.41,1027384.96,0.0000,79.012346,19.262346,17.802298,-555.862826,1771.561,1.041357e+09,0.000000,702.331962,-84.540295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,14.777778,13.777778,14.7,998.6,1.06,17.222222,12.222222,218.382716,189.827161,216.09,997201.96,1.1236,296.604938,149.382716,3227.211250,2615.396435,3176.523,9.958059e+08,1.191016,5108.196157,1825.788751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,23.500000,16.833333,14.3,1020.0,0.00,30.611111,17.777778,552.250000,283.361111,204.49,1040400.00,0.0000,937.040123,316.049383,12977.875000,4769.912034,2924.207,1.061208e+09,0.000000,28683.839332,5618.655695,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


We can do the same with the test set:

In [508]:
X_test_numerical_powers = np.hstack((X_test_df[numerical_columns]**(i+1) for i in range(3)))
X_test_np_powers = np.concatenate((X_test_numerical_powers,X_test_df.drop(numerical_columns, axis=1)),axis=1)
X_test_df_powers = pd.DataFrame(X_test_np_powers)
newcolname = ['Temp', 'Dewpoint', 'Windspeed', 'Pressure', 'Precipitation', 'TMAX_C', 'TMIN_C', 'Temp^2', 'Dewpoint^2', 'Windspeed^2', 'Pressure^2', 'Precipitation^2', 'TMAX_C^2', 'TMIN_^2','Temp^3', 'Dewpoint^3', 'Windspeed^3', 'Pressure^3', 'Precipitation^3', 'TMAX_C^3', 'TMIN_C^3'] + list(X_train_df.drop(numerical_columns, axis=1))
X_test_df_powers.columns = newcolname
X_test_df_powers

,Temp,Dewpoint,Windspeed,Pressure,Precipitation,TMAX_C,TMIN_C,Temp^2,Dewpoint^2,Windspeed^2,Pressure^2,Precipitation^2,TMAX_C^2,TMIN_^2,Temp^3,Dewpoint^3,Windspeed^3,Pressure^3,Precipitation^3,TMAX_C^3,TMIN_C^3,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12,Weekend_1,Season_Spring,Season_Summer,Season_Winter,DOW_Monday,DOW_Saturday,DOW_Sunday,DOW_Thursday,DOW_Tuesday,DOW_Wednesday
0,19.166667,12.611111,7.7,1024.5,0.00,26.111111,14.388889,367.361111,159.040123,59.29,1049600.25,0.0000,681.790123,207.040123,7041.087967,2005.672668,456.533,1.075315e+09,0.000000,17802.297666,2979.077333,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,-4.166667,-15.722222,9.0,1023.6,0.00,-1.111111,-7.777778,17.361111,247.188272,81.00,1047756.96,0.0000,1.234568,60.493827,-72.337963,-3886.348935,729.000,1.072484e+09,0.000000,-1.371742,-470.507545,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2.111111,-4.833333,6.3,1009.6,0.00,5.611111,-0.500000,4.456790,23.361111,39.69,1019292.16,0.0000,31.484568,0.250000,9.408779,-112.912037,250.047,1.029077e+09,0.000000,176.663409,-0.125000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-1.944444,-11.666667,15.3,1009.1,0.01,6.111111,-3.888889,3.780864,136.111111,234.09,1018282.81,0.0001,37.345679,15.123457,-7.351680,-1587.962964,3581.577,1.027549e+09,0.000001,228.223594,-58.813443,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-4.777778,-10.388889,7.7,1019.3,0.91,0.611111,-7.222222,22.827160,107.929012,59.29,1038972.49,0.8281,0.373457,52.160494,-109.063100,-1121.262518,456.533,1.059025e+09,0.753571,0.228224,-376.714678,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,17.611111,9.777778,3.2,1017.8,0.00,26.111111,15.000000,310.151235,95.604938,10.24,1035916.84,0.0000,681.790123,225.000000,5462.107852,934.803841,32.768,1.054356e+09,0.000000,17802.297666,3375.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-0.444444,-4.166667,9.4,1017.8,0.09,3.277778,-2.222222,0.197531,17.361111,88.36,1035916.84,0.0081,10.743827,4.938272,-0.087791,-72.337963,830.584,1.054356e+09,0.000729,35.215878,-10.973937,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,10.166667,0.555556,11.0,1011.9,0.03,13.888889,2.777778,103.361111,0.308642,121.00,1023941.61,0.0009,192.901235,7.716049,1050.837964,0.171468,1331.000,1.036127e+09,0.000027,2679.183814,21.433471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,10.000000,3.111111,11.1,1027.2,0.00,17.777778,3.277778,100.000000,9.679012,123.21,1055139.84,0.0000,316.049383,10.743827,1000.000000,30.112483,1367.631,1.083840e+09,0.000000,5618.655695,35.215878,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,13.388889,7.666667,7.3,1018.7,0.00,18.888889,6.111111,179.262346,58.777778,53.29,1037749.69,0.0000,356.790123,37.345679,2400.123629,450.629630,389.017,1.057156e+09,0.000000,6739.369000,228.223594,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We can do forward and backward selection as well on this new data frame. 

In [509]:
predictors_forward = step_forwards_backwards(X_train_df_powers, y_train, direction='forward')
predictors_forward

['DOW_Sunday',
 'DOW_Monday',
 'Temp',
 'DOW_Wednesday',
 'Month_12',
 'Weekend_1',
 'Windspeed^3',
 'DOW_Thursday',
 'Month_9',
 'DOW_Tuesday',
 'Dewpoint^3']

In [515]:
predictors_backward = step_forwards_backwards(X_train_df_powers, y_train, direction='backward')
predictors_backward

['DOW_Wednesday',
 'DOW_Thursday',
 'Weekend_1',
 'Precipitation',
 'DOW_Monday',
 'DOW_Saturday',
 'Month_9',
 'DOW_Tuesday',
 'Temp',
 'Windspeed^3']

In [516]:
X = sm.add_constant(X_train_df_powers[predictors_backward])
X_test = sm.add_constant(X_test_df_powers[predictors_backward])
y = train['Incidence'].values.reshape(-1,1)

model = OLS(endog=y, exog=X)
result = model.fit()

y_hat_train = result.predict()
y_hat_test = result.predict(exog=X_test)

print('Forward Selection Training R2 = ', r2_score(y_train, y_hat_train))
print('Forward Selection Testing R2 = ', r2_score(y_test, y_hat_test))

Forward Selection Training R2 =  0.414915602521
Forward Selection Testing R2 =  0.361369756369


In [517]:
X = sm.add_constant(X_train_df_powers[predictors_forward])
X_test = sm.add_constant(X_test_df_powers[predictors_forward])
y = train['Incidence'].values.reshape(-1,1)

model = OLS(endog=y, exog=X)
result = model.fit()

y_hat_train = result.predict()
y_hat_test = result.predict(exog=X_test)

print('Forward Selection Training R2 = ', r2_score(y_train, y_hat_train))
print('Forward Selection Testing R2 = ', r2_score(y_test, y_hat_test))

Forward Selection Training R2 =  0.422023570329
Forward Selection Testing R2 =  0.374213258131


## Part (4) Create the expanded matrix containing all terms

Let's now create a design matrix that includes all polynomial terms up to the third order, including all interactions. 

In [518]:
all_poly_terms = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)

X_train_full_poly = all_poly_terms.fit_transform(X_train_df)
X_test_full_poly = all_poly_terms.fit_transform(X_test_df)

print('number of total predictors', X_train_full_poly.shape[1])


number of total predictors 4494


In [519]:
X_train_full_poly.shape

(876, 4494)

We can use the following function to get an idea of the interactions between the variables.

In [520]:
all_poly_terms.get_feature_names()

['x0',
 'x1',
 'x2',
 'x3',
 'x4',
 'x5',
 'x6',
 'x7',
 'x8',
 'x9',
 'x10',
 'x11',
 'x12',
 'x13',
 'x14',
 'x15',
 'x16',
 'x17',
 'x18',
 'x19',
 'x20',
 'x21',
 'x22',
 'x23',
 'x24',
 'x25',
 'x26',
 'x27',
 'x0^2',
 'x0 x1',
 'x0 x2',
 'x0 x3',
 'x0 x4',
 'x0 x5',
 'x0 x6',
 'x0 x7',
 'x0 x8',
 'x0 x9',
 'x0 x10',
 'x0 x11',
 'x0 x12',
 'x0 x13',
 'x0 x14',
 'x0 x15',
 'x0 x16',
 'x0 x17',
 'x0 x18',
 'x0 x19',
 'x0 x20',
 'x0 x21',
 'x0 x22',
 'x0 x23',
 'x0 x24',
 'x0 x25',
 'x0 x26',
 'x0 x27',
 'x1^2',
 'x1 x2',
 'x1 x3',
 'x1 x4',
 'x1 x5',
 'x1 x6',
 'x1 x7',
 'x1 x8',
 'x1 x9',
 'x1 x10',
 'x1 x11',
 'x1 x12',
 'x1 x13',
 'x1 x14',
 'x1 x15',
 'x1 x16',
 'x1 x17',
 'x1 x18',
 'x1 x19',
 'x1 x20',
 'x1 x21',
 'x1 x22',
 'x1 x23',
 'x1 x24',
 'x1 x25',
 'x1 x26',
 'x1 x27',
 'x2^2',
 'x2 x3',
 'x2 x4',
 'x2 x5',
 'x2 x6',
 'x2 x7',
 'x2 x8',
 'x2 x9',
 'x2 x10',
 'x2 x11',
 'x2 x12',
 'x2 x13',
 'x2 x14',
 'x2 x15',
 'x2 x16',
 'x2 x17',
 'x2 x18',
 'x2 x19',
 'x2 x20',
 '

In [521]:
X_train_full_poly

array([[ 24.88888889,  21.16666667,   7.2       , ...,   0.        ,
          0.        ,   0.        ],
       [ -4.33333333, -15.83333333,  12.8       , ...,   0.        ,
          0.        ,   0.        ],
       [  6.05555556,   3.88888889,   5.6       , ...,   0.        ,
          0.        ,   0.        ],
       ..., 
       [  9.61111111,   1.55555556,  10.3       , ...,   0.        ,
          0.        ,   0.        ],
       [ 14.94444444,   4.5       ,  10.9       , ...,   0.        ,
          0.        ,   0.        ],
       [ 14.88888889,   3.27777778,   5.7       , ...,   0.        ,
          0.        ,   0.        ]])

If we wanted to drop the 0's, we can use the following function:

In [522]:
zero_column_index = np.where(~X_train_full_poly.any(axis=0))[0]

In [523]:
X_train_full_poly_nonzero_col = np.delete(X_train_full_poly, zero_column_index, axis = 1)
X_test_full_poly_nonzero_col = np.delete(X_test_full_poly, zero_column_index, axis = 1)

print(X_train_full_poly_nonzero_col)

[[ 24.88888889  21.16666667   7.2        ...,   0.           0.           0.        ]
 [ -4.33333333 -15.83333333  12.8        ...,   0.           0.           0.        ]
 [  6.05555556   3.88888889   5.6        ...,   0.           0.           0.        ]
 ..., 
 [  9.61111111   1.55555556  10.3        ...,   0.           0.           0.        ]
 [ 14.94444444   4.5         10.9        ...,   0.           0.           0.        ]
 [ 14.88888889   3.27777778   5.7        ...,   0.           0.           0.        ]]


Now we can fit our PCA model:

In [524]:
pca = PCA(n_components=5)
pca.fit(X_train_full_poly)
train_pca = pca.transform(X_train_full_poly)
test_pca = pca.transform(X_test_full_poly)

print('Explained variance ratio:', pca.explained_variance_ratio_)


Explained variance ratio: [ 0.61152415  0.35761929  0.01389551  0.0082353   0.00516814]


We can obtain the corresponding coefficients of each principal component.

In [525]:
pca.components_
# First "row" of array corresponds to first component weights. 

array([[ -2.42699108e-07,  -2.73316890e-07,  -1.87865727e-08, ...,
         -0.00000000e+00,  -0.00000000e+00,  -4.24431891e-10],
       [ -4.07296529e-07,  -4.22851533e-07,   7.55066168e-08, ...,
         -0.00000000e+00,  -0.00000000e+00,   6.72308095e-10],
       [  1.76716058e-07,  -6.55414859e-07,   3.91924831e-07, ...,
          0.00000000e+00,   0.00000000e+00,  -3.59013804e-09],
       [ -2.75401712e-08,   3.26999301e-07,   8.85140001e-07, ...,
          0.00000000e+00,   0.00000000e+00,   3.19735439e-09],
       [ -1.56834886e-08,   3.47704356e-07,  -2.99562697e-08, ...,
          0.00000000e+00,   0.00000000e+00,   1.22520679e-09]])

If you recall, our weights squared had to sum to one. Let's see if this is the case:

In [429]:
(pca.components_ ** 2).sum(axis = 1)

array([ 1.,  1.,  1.,  1.,  1.])

Now we will look at each of the components, or weights from our 5 principal components. Because we centered our dataset, each value is essentially a correlation value. 

In [527]:
feature_frame = pd.DataFrame(pca.components_,columns=all_poly_terms.get_feature_names(),index = ['PC-1','PC-2','PC-3','PC-4','PC-5'])
feature_frame

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x0^2,x0 x1,x0 x2,x0 x3,x0 x4,x0 x5,x0 x6,x0 x7,x0 x8,x0 x9,x0 x10,x0 x11,x0 x12,x0 x13,x0 x14,x0 x15,x0 x16,x0 x17,x0 x18,x0 x19,x0 x20,x0 x21,x0 x22,x0 x23,x0 x24,x0 x25,x0 x26,x0 x27,x1^2,x1 x2,x1 x3,x1 x4,x1 x5,x1 x6,x1 x7,x1 x8,x1 x9,x1 x10,x1 x11,x1 x12,x1 x13,x1 x14,x1 x15,x1 x16,x1 x17,x1 x18,x1 x19,x1 x20,x1 x21,x1 x22,x1 x23,x1 x24,x1 x25,x1 x26,x1 x27,x2^2,x2 x3,x2 x4,x2 x5,x2 x6,x2 x7,x2 x8,x2 x9,x2 x10,x2 x11,x2 x12,x2 x13,x2 x14,x2 x15,x2 x16,x2 x17,x2 x18,x2 x19,x2 x20,x2 x21,x2 x22,x2 x23,x2 x24,x2 x25,x2 x26,x2 x27,x3^2,x3 x4,x3 x5,x3 x6,x3 x7,x3 x8,x3 x9,x3 x10,x3 x11,x3 x12,x3 x13,x3 x14,x3 x15,x3 x16,x3 x17,x3 x18,x3 x19,x3 x20,x3 x21,x3 x22,x3 x23,x3 x24,x3 x25,x3 x26,x3 x27,x4^2,x4 x5,x4 x6,x4 x7,x4 x8,x4 x9,x4 x10,x4 x11,x4 x12,x4 x13,x4 x14,x4 x15,x4 x16,x4 x17,x4 x18,x4 x19,x4 x20,x4 x21,x4 x22,x4 x23,x4 x24,x4 x25,x4 x26,x4 x27,x5^2,x5 x6,x5 x7,x5 x8,x5 x9,x5 x10,x5 x11,x5 x12,x5 x13,x5 x14,x5 x15,x5 x16,x5 x17,x5 x18,x5 x19,x5 x20,x5 x21,x5 x22,x5 x23,x5 x24,x5 x25,x5 x26,x5 x27,x6^2,x6 x7,x6 x8,x6 x9,x6 x10,x6 x11,x6 x12,x6 x13,x6 x14,x6 x15,x6 x16,x6 x17,x6 x18,x6 x19,x6 x20,x6 x21,x6 x22,x6 x23,x6 x24,x6 x25,x6 x26,x6 x27,x7^2,x7 x8,x7 x9,x7 x10,x7 x11,x7 x12,x7 x13,x7 x14,x7 x15,x7 x16,x7 x17,x7 x18,x7 x19,x7 x20,x7 x21,x7 x22,x7 x23,x7 x24,x7 x25,x7 x26,x7 x27,x8^2,x8 x9,x8 x10,x8 x11,x8 x12,x8 x13,x8 x14,x8 x15,x8 x16,x8 x17,x8 x18,x8 x19,x8 x20,x8 x21,x8 x22,x8 x23,x8 x24,x8 x25,x8 x26,x8 x27,x9^2,x9 x10,x9 x11,x9 x12,x9 x13,x9 x14,x9 x15,x9 x16,x9 x17,x9 x18,x9 x19,x9 x20,x9 x21,x9 x22,x9 x23,x9 x24,x9 x25,x9 x26,x9 x27,x10^2,x10 x11,x10 x12,x10 x13,x10 x14,x10 x15,x10 x16,x10 x17,x10 x18,x10 x19,x10 x20,x10 x21,x10 x22,x10 x23,x10 x24,x10 x25,x10 x26,x10 x27,x11^2,x11 x12,x11 x13,x11 x14,x11 x15,x11 x16,x11 x17,x11 x18,x11 x19,x11 x20,x11 x21,x11 x22,x11 x23,x11 x24,x11 x25,x11 x26,x11 x27,x12^2,x12 x13,x12 x14,x12 x15,x12 x16,x12 x17,x12 x18,x12 x19,x12 x20,x12 x21,x12 x22,x12 x23,x12 x24,x12 x25,x12 x26,x12 x27,x13^2,x13 x14,x13 x15,x13 x16,x13 x17,x13 x18,x13 x19,x13 x20,x13 x21,x13 x22,x13 x23,x13 x24,x13 x25,x13 x26,x13 x27,x14^2,x14 x15,x14 x16,x14 x17,x14 x18,x14 x19,x14 x20,x14 x21,x14 x22,x14 x23,x14 x24,x14 x25,x14 x26,x14 x27,x15^2,x15 x16,x15 x17,x15 x18,x15 x19,x15 x20,x15 x21,x15 x22,x15 x23,x15 x24,x15 x25,x15 x26,x15 x27,x16^2,x16 x17,x16 x18,x16 x19,x16 x20,x16 x21,x16 x22,x16 x23,x16 x24,x16 x25,x16 x26,x16 x27,x17^2,x17 x18,x17 x19,x17 x20,x17 x21,x17 x22,x17 x23,x17 x24,x17 x25,x17 x26,x17 x27,x18^2,x18 x19,x18 x20,x18 x21,x18 x22,x18 x23,x18 x24,x18 x25,x18 x26,x18 x27,x19^2,x19 x20,x19 x21,x19 x22,x19 x23,x19 x24,x19 x25,x19 x26,x19 x27,x20^2,x20 x21,x20 x22,x20 x23,x20 x24,x20 x25,x20 x26,x20 x27,x21^2,x21 x22,x21 x23,x21 x24,x21 x25,x21 x26,x21 x27,x22^2,x22 x23,x22 x24,x22 x25,x22 x26,x22 x27,x23^2,x23 x24,x23 x25,x23 x26,x23 x27,x24^2,x24 x25,x24 x26,x24 x27,x25^2,x25 x26,x25 x27,x26^2,x26 x27,x27^2,x0^3,x0^2 x1,x0^2 x2,x0^2 x3,x0^2 x4,x0^2 x5,x0^2 x6,x0^2 x7,x0^2 x8,x0^2 x9,x0^2 x10,x0^2 x11,x0^2 x12,x0^2 x13,x0^2 x14,x0^2 x15,x0^2 x16,x0^2 x17,x0^2 x18,x0^2 x19,x0^2 x20,x0^2 x21,x0^2 x22,x0^2 x23,x0^2 x24,x0^2 x25,x0^2 x26,x0^2 x27,x0 x1^2,x0 x1 x2,x0 x1 x3,x0 x1 x4,x0 x1 x5,x0 x1 x6,x0 x1 x7,x0 x1 x8,x0 x1 x9,x0 x1 x10,x0 x1 x11,x0 x1 x12,x0 x1 x13,x0 x1 x14,x0 x1 x15,x0 x1 x16,x0 x1 x17,x0 x1 x18,x0 x1 x19,x0 x1 x20,x0 x1 x21,x0 x1 x22,x0 x1 x23,x0 x1 x24,x0 x1 x25,x0 x1 x26,x0 x1 x27,x0 x2^2,x0 x2 x3,x0 x2 x4,x0 x2 x5,x0 x2 x6,x0 x2 x7,x0 x2 x8,x0 x2 x9,x0 x2 x10,x0 x2 x11,...,x14 x19 x20,x14 x19 x21,x14 x19 x22,x14 x19 x23,x14 x19 x24,x14 x19 x25,x14 x19 x26,x14 x19 x27,x14 x20^2,x14 x20 x21,x14 x20 x22,x14 x20 x23,x14 x20 x24,x14 x20 x25,x14 x20 x26,x14 x20 x27,x14 x21^2,x14 x21 x22,x14 x21 x23,x14 x21 x24,x14 x21 x25,x14 x21 x26,x14 x21 x27,x14 x22^2,x14 x22 x23,x14 x22 x24,x14 x22 x25,x14 x22 x26,x14 x22 x27,x14 x23^2,x14 x23 x24,x14 x23 x25,x14 x23 x26,x14

Now, let's take a look at the top correlation values for the first partial component. 

In [426]:
feature_frame[0:1].sort_values(by=['PC-1'], axis = 1, ascending = False).T[0:3]

,PC-1
x4^3,0.960024
x2 x4^2,0.146450
x4^2,0.101705


The first partial component really depends on x4^3, x2 x4^2 and x4^2. When each of these values increase, we would expect each of them to increase together.

Finally, let's look at the R-squared models fo the first 10 partial components. 

In [529]:

R2_pca_train = []
R2_pca_test = []

for i in range(10):
    pca = PCA(n_components=i+1)
    pca.fit(X_train_full_poly_nonzero_col)
    
    X_train_pca = pca.transform(X_train_full_poly_nonzero_col)
    X_test_pca = pca.transform(X_test_full_poly_nonzero_col)
    
    regression_model = LinearRegression(fit_intercept=True) 
    regression_model.fit(X_train_pca, y_train)
    
    R2_pca_train.append(regression_model.score(X_train_pca, y_train)) 
    R2_pca_test.append(regression_model.score(X_test_pca, y_test)) 
    
    print('Explained variance ratio for Model with',i+1, 'components:', pca.explained_variance_ratio_)
    

Explained variance ratio for Model with 1 components: [ 0.61152415]
Explained variance ratio for Model with 2 components: [ 0.61152415  0.35761929]
Explained variance ratio for Model with 3 components: [ 0.61152415  0.35761929  0.01389551]
Explained variance ratio for Model with 4 components: [ 0.61152415  0.35761929  0.01389551  0.0082353 ]
Explained variance ratio for Model with 5 components: [ 0.61152415  0.35761929  0.01389551  0.0082353   0.00516814]
Explained variance ratio for Model with 6 components: [ 0.61152415  0.35761929  0.01389551  0.0082353   0.00516814  0.00117775]
Explained variance ratio for Model with 7 components: [  6.11524146e-01   3.57619291e-01   1.38955062e-02   8.23529885e-03
   5.16813644e-03   1.17774957e-03   3.67690930e-04]
Explained variance ratio for Model with 8 components: [  6.11524146e-01   3.57619291e-01   1.38955062e-02   8.23529885e-03
   5.16813644e-03   1.17774957e-03   3.67690930e-04   3.48066309e-04]
Explained variance ratio for Model with 9 c

In [531]:
R2_pca_train

[0.057308247917507993,
 0.20051997215827921,
 0.20235152960663594,
 0.22208195460101976,
 0.22208468045708182,
 0.2288197135038825,
 0.25191914232007251,
 0.27554572502076669,
 0.27571221936073886,
 0.27683252409882131]

In [532]:
R2_pca_test

[0.019630405030818476,
 0.13097374666377815,
 0.13887240964171155,
 0.15502674205484579,
 0.15531952268930868,
 0.16085901610608569,
 0.17421211674861481,
 0.21927465931988643,
 0.21979176294449454,
 0.22367708504109418]